# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("..\output_data\cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ulladulla,-35.3500,150.4667,23.20,70,32,4.39,AU,1705904145
1,1,waitangi,-43.9535,-176.5597,18.79,92,100,4.92,NZ,1705904145
2,2,afaahiti,-17.7500,-149.2833,22.93,81,100,7.91,PF,1705904146
3,3,konosha,60.9736,40.2570,-10.83,92,100,4.35,RU,1705904146
4,4,jamestown,42.0970,-79.2353,-12.55,93,0,3.09,US,1705904128


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c="Humidity",
    geo= True,
    tiles= "OSM",
    colorbar= True,
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_cities = city_data_df.loc[(city_data_df['Max Temp'] <30) &
                                   (city_data_df['Max Temp'] >20) &
                                   (city_data_df['Wind Speed'] <4.5) & (city_data_df['Cloudiness'] <=0)]

# Drop any rows with null values
narrowed_down_cities = narrowed_cities.dropna()

# Display sample data
narrowed_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,matoury,4.8483,-52.3317,25.98,89,0,3.60,GF,1705904155
156,156,jeddah,21.5169,39.2192,26.92,50,0,1.54,SA,1705904242
242,242,juan griego,11.0817,-63.9655,26.07,89,0,3.60,VE,1705904296
250,250,gebeit,21.0667,36.3167,20.43,69,0,0.82,SD,1705904301
326,326,arica,-18.4750,-70.3042,22.13,83,0,1.03,CL,1705904350
378,378,alo,7.4178,6.7371,20.47,36,0,1.44,NG,1705904382
438,438,khark,29.2614,50.3306,21.01,56,0,0.00,IR,1705904419
451,451,coahuayana de hidalgo,18.7000,-103.6583,22.20,82,0,1.79,MX,1705904428
525,525,remire-montjoly,4.9167,-52.2667,26.02,89,0,3.60,GF,1705904474
570,570,richards bay,-28.7830,32.0377,24.82,72,0,1.06,ZA,1705904504


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_cities.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,matoury,4.8483,-52.3317,25.98,89,0,3.60,GF,1705904155,
156,156,jeddah,21.5169,39.2192,26.92,50,0,1.54,SA,1705904242,
242,242,juan griego,11.0817,-63.9655,26.07,89,0,3.60,VE,1705904296,
250,250,gebeit,21.0667,36.3167,20.43,69,0,0.82,SD,1705904301,
326,326,arica,-18.4750,-70.3042,22.13,83,0,1.03,CL,1705904350,
378,378,alo,7.4178,6.7371,20.47,36,0,1.44,NG,1705904382,
438,438,khark,29.2614,50.3306,21.01,56,0,0.00,IR,1705904419,
451,451,coahuayana de hidalgo,18.7000,-103.6583,22.20,82,0,1.79,MX,1705904428,
525,525,remire-montjoly,4.9167,-52.2667,26.02,89,0,3.60,GF,1705904474,
570,570,richards bay,-28.7830,32.0377,24.82,72,0,1.06,ZA,1705904504,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
params

{'limit': 1,
 'filter': 'radius@1000,16.8634,-99.8901',
 'bias': 'proximity@16.8634,-99.8901'}

In [20]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "limit": 1,
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"radius@{radius},{lat},{lon}"
    params["bias"] = f"proximity@{lat},{lon}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
matoury - nearest hotel: No hotel found
jeddah - nearest hotel: No hotel found
juan griego - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
arica - nearest hotel: No hotel found
alo - nearest hotel: No hotel found
khark - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,matoury,4.8483,-52.3317,25.98,89,0,3.60,GF,1705904155,No hotel found
156,156,jeddah,21.5169,39.2192,26.92,50,0,1.54,SA,1705904242,No hotel found
242,242,juan griego,11.0817,-63.9655,26.07,89,0,3.60,VE,1705904296,No hotel found
250,250,gebeit,21.0667,36.3167,20.43,69,0,0.82,SD,1705904301,No hotel found
326,326,arica,-18.4750,-70.3042,22.13,83,0,1.03,CL,1705904350,No hotel found
378,378,alo,7.4178,6.7371,20.47,36,0,1.44,NG,1705904382,No hotel found
438,438,khark,29.2614,50.3306,21.01,56,0,0.00,IR,1705904419,No hotel found
451,451,coahuayana de hidalgo,18.7000,-103.6583,22.20,82,0,1.79,MX,1705904428,No hotel found
525,525,remire-montjoly,4.9167,-52.2667,26.02,89,0,3.60,GF,1705904474,No hotel found
570,570,richards bay,-28.7830,32.0377,24.82,72,0,1.06,ZA,1705904504,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c="Humidity",
    geo= True,
    tiles= "OSM",
    colorbar= True,
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)